In [1]:
# ---------------------------------------- #
# --------------- importing -------------- #
# ---------------------------------------- #
import pandas as pd
import numpy as np
import copy

# 0. 데이터 업로드 및 전처리

In [2]:
# ---------------------------------------- #
# ------------- 데이터 전처리 ------------ #
# ---------------------------------------- #

"""
* pandas & numpy  이용

* 행과 열 정리된 csv 파일 불러오는 것으로 변경

* 인덱스 정리
"""

# 데이터 불러오기
df = pd.read_csv("./global_weekly_top_200.csv", index_col=0)

# 인덱스 정리
df = df.reset_index()
df = df.drop(['Position'],axis=1)
df = df.drop(['URL'],axis=1)
df = df.drop(['index'],axis=1)
df.loc[-1] = [0, 0, 0]  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 

# 1. 관리자

## 1.1. 데이터 구조 및 주요 기능

In [3]:
# ---------------------------------------- #
# -------------- 관리자 모드 ------------- #
# ---------------------------------------- #

"""
* 데이터 구조
  - 이진트리 구조를 사용한 최대힙
  - upHeap, downHeap

* 주요 기능
  - Create: 삽입 
  - Search: 검색
  - Update: 업데이트
  - Delete: 삭제
  - Print: 출력
"""

##### 관리자 모드: 데이터 구조 및 주요 기능 #####
class Heap:
  def __init__(self, df_data):
    self.df = df_data.copy()

  def bHeap(self):
    for i in range(len(self.df)//2, 0, -1):
      self.downHeap(self.df.iloc[i])

  # upHeap 수행: "Streams" 기준
  def upHeap(self, p):    
    if p['Streams'] == self.df.iloc[1]['Streams']:
      return
    for i, data in self.df.iterrows():
      if data['Streams'] == p['Streams']:
          p_index = i
    if p['Streams'] < self.df.iloc[p_index//2]['Streams']:
        return
    self.df.iloc[p_index] = self.df.iloc[p_index//2]
    self.df.iloc[p_index//2] = p
    self.upHeap(self.df.iloc[p_index//2])

  # downHeap 수행: "Streams" 기준
  def downHeap(self, p):
    for i, data in self.df.iterrows():
      if data['Streams'] == p['Streams']:
          p_index = i
    if (len(self.df) <= p_index*2) & (len(self.df) <= (p_index*2+1)):
        return
    b = self.df.iloc[p_index*2]
    if (len(self.df) > p_index*2+1):
        if (self.df.iloc[p_index*2+1]['Streams'] > b['Streams']):
          b = self.df.iloc[p_index*2+1]
    if p['Streams'] > b['Streams']:
        return
    for i, data in self.df.iterrows():
        if data['Streams'] == b['Streams']:
          b_index = i
    self.df.iloc[p_index] = self.df.iloc[b_index]
    self.df.iloc[b_index] = p
    self.downHeap(self.df.iloc[b_index])

  # 삽입1: 초기 Heap 구현 및 insert_2에서 호출
  def insert(self, row):
    self.df = self.df.append(row,ignore_index=True)
    p = self.df.iloc[-1]
    self.upHeap(p)

  # 삽입2: main 함수를 통한 삽입
  def insert_2(self):
    try:
      Track_Name = input("노래 제목을 입력해주세요. ")
      Artist_Name = input("가수 이름을 입력해주세요. ")
      Streams = int(input("스트리밍 수를 입력해주세요. "))
      
      self.insert(pd.DataFrame([[Track_Name, Artist_Name, Streams]], columns=['Track Name', 'Artist', 'Streams']))
      print("\n###삽입 결과###\n")
      print(f" \n Track Name: {Track_Name}\n Artist: {Artist_Name} \n Streams: {Streams} \n")
      print("\ninsert 완료\n")
    
    except:
      print("잘못된 값을 입력하셨습니다.\n")
      return

  # 검색: 검색어를 입력하면 Track Name 혹은 Artist에 해당 검색어를 포함하는 음원 출력
  def search(self):
    keyword = input("원하는 검색어를 입력해주세요. ")

    try:
      df_name = self.df[self.df["Track Name"].str.contains(keyword, case=False, na=False)]
      df_artist = self.df[self.df["Artist"].str.contains(keyword, case=False, na=False)]
      df_result = pd.concat([df_name, df_artist])
      df_result = df_result.sort_index()

      if len(df_result)==0: ## 검색한 결과가 없으면 
        print("결과가 존재하지 않습니다.")
        return False
      
      print("\n###검색 결과###")
      for i, data in df_result.iterrows():
        print(f"{i}번째 음원: \n Track Name: {data['Track Name']} \n Artist: {data['Artist']} \n")
      return True

    except:
      print("잘못된 값을 입력하셨습니다.")
      return False

  # 업데이트: Track Name, Artist, Streams 모두 업데이트 가능
  def update(self):
    result = self.search()

    if not result:
      return

    try:
      select = int(input("몇 번째 노래를 업데이트할까요? "))
      part = input("어느 부분을 업데이트할까요?[Track Name, Artist, Streams]: ")
      if part not in ["Track Name", "Artist", "Streams"]:
        print("잘못된 값을 입력하셨습니다.")
        return
      
      if (part == 'Streams'):
        value = int(input("어느 값으로 업데이트할까요? "))
        self.df.loc[self.df.index == select, ('Streams')] = value
        p = self.df.iloc[select]
        print(p)
        self.upHeap(p)
        for i, data in self.df.iterrows():
          if data['Streams'] == p['Streams']:
            select = i
      else:
        value = input("어느 값으로 업데이트할까요? ")
        self.df.loc[self.df.index == select, (part)] = value
      df_sample_4 = self.df.iloc[[select], :]

    except:
      print("잘못된 값을 입력하셨습니다.")
      return
    
    print("\n###업데이트 결과###")
    for i, data in df_sample_4.iterrows():
      print(f" Track Name: {data['Track Name']}\n Artist: {data['Artist']} \n Streams: {data['Streams']} \n")
    print("\nupdate 완료")

  # 삭제: 검색 후 번호를 입력하여 삭제
  def delete(self):
    result = self.search()
    if not result:
      return

    try:
      n = int(input("몇 번째 노래를 삭제할까요? "))
      root = self.df.iloc[n]
      temp = self.df.iloc[-1]
      self.df.drop(index=n, axis=0)
      self.df.iloc[n]= self.df.iloc[-1]
      self.df = self.df.iloc[:-1 , :]
      self.downHeap(self.df.iloc[n])
      print("\ndelete 완료")

    except:
      print("잘못된 값을 입력하셨습니다.")
  
  # 상위 n개 음원 출력: Root 노드 출력 후 최대힙 시행
  def pr(self):
    music = []
    try:
      n = int(input("몇 개의 노래를 보여드릴까요? "))

      for data in range(n):
        root = self.df.iloc[1]
        temp = self.df.iloc[-1]
        print(f"\nTrack Name: {root['Track Name']}\nArtist: {root['Artist']}\nStreams: {root['Streams']}")
        music.append([root["Track Name"], root["Artist"], root["Streams"]])

        self.df.drop(index=1, axis=0)
        self.df.iloc[1]= self.df.iloc[-1]
        self.df = self.df.iloc[:-1 , :]
        self.downHeap(self.df.iloc[1])

      for i in range(n):
        self.insert(pd.DataFrame([[music[i][0], music[i][1], music[i][2]]], columns=['Track Name', 'Artist', 'Streams']))
          
    except:
      print("잘못된 값을 입력하셨습니다.")

## 1.2. admin_main

In [4]:
##### 관리자 모드: 실행 #####
def admin_main():
  while (1):
    print("*"*30)
    print(" 1: 추가 \n 2: 업데이트 \n 3: 검색 \n 4: 제거 \n 5: 출력 \n q: 종료")
    print("*"*30)
    function = input("무엇을 하고자 하십니까? ")
    
    if function == "q":
      break
    elif function == "1":
      DB.insert_2()
    elif function == "2":
      DB.update()
    elif function == "3":
      DB.search()
    elif function == "4":
      DB.delete()
    elif function == "5": # 삭제
      DB.pr()
      
    else:
      print("잘못된 값을 입력하셨습니다.")


# 2. 이용자

## 2.1. Ranking 데이터

In [5]:
# ---------------------------------------- #
# ------------ Ranking 데이터 ------------ #
# ---------------------------------------- #

"""
* 관리자 모드의 힙을 바탕으로 최대힙 정렬을 시행하여 Ranking 데이터를 구축
* 이용자에게 제공
"""

# Ranking 데이터 구축
def Create_Ranking():
  raw = copy.deepcopy(DB)
  raw.insert(pd.DataFrame([[0, 0, 0]], columns=['Track Name', 'Artist', 'Streams']))

  Ranking = pd.DataFrame(columns=["Track Name", "Artist"])

  rank_list = []

  # 최대힙 정렬
  for i in range(len(raw.df)-2):
    root = raw.df.iloc[1]
    temp = raw.df.iloc[-1]
    Ranking = Ranking.append({"Track Name":root["Track Name"], "Artist":root["Artist"]}, ignore_index=True)

    raw.df.drop(index=1, axis=0)
    raw.df.iloc[1]=raw.df.iloc[-1]
    raw.df = raw.df.iloc[:-1 , :]
    raw.downHeap(raw.df.iloc[1])

  return Ranking

## 2.2. 데이터 구조

In [6]:
# ---------------------------------------- #
# -------------- 이용자 모드 ------------- #
# ---------------------------------------- #

"""
* 데이터 구조
  - 플레이리스트
  - 이용자들이 원하는 음원을 자유롭게 추가/삭제/재생
  - 배열로 구현한 원형큐

* 주요 기능
  - Display_Ranking: Ranking 데이터 출력
  - Search_Ranking: Ranking 데이터 검색
  - Search_PlayList: 플레이리스트 검색
  - Create: 플레이리스트 음원 추가
  - Play: 플레이리스트 재생
  - Delete: 플레이리스트 음원 제거
"""

##### 이용자 모드: 데이터 구조 #####
class PlayList:   
  def __init__(self):
    self.front = 0
    self.rear = 0
    self.n = 0
    self.array = []
  
  # isEmpty: PlayList가 비어 있는지 확인
  def isEmpty(self):
    return self.front == self.rear
  
  # enqueue: PlayList의 rear에 음원 삽입
  def enqueue(self, item):
    tmp = item
    self.n = self.n + 1
    self.rear = self.rear + 1
    self.array.append(item)
    return tmp
  
  # dequeue: PlayList의 index에 있는 음원 제거 후 반환
  def dequeue(self, index):
    tmp = self.array[index]
    del self.array[index]
    self.rear = self.rear - 1
    self.n = self.n - 1
    return tmp

## 2.3. 주요 기능

In [7]:
##### 이용자 모드: 주요 기능 #####
# Ranking 데이터 전부 출력하기
def Display_Ranking(Ranking):
  for i, data in Ranking.iterrows():
    print(f"{i+1:3}위 \n Track Name: {data['Track Name']} \n Artist: {data['Artist']} \n")


# Ranking 데이터 검색
def Search_Ranking(Ranking):
    keyword = input("원하는 검색어를 입력해주세요. ")

    try:
      df_name = Ranking[Ranking["Track Name"].str.contains(keyword, case=False, na=False)]
      df_artist = Ranking[Ranking["Artist"].str.contains(keyword, case=False, na=False)]
      df_result = pd.concat([df_name, df_artist])
      df_result = df_result.sort_index()

      if len(df_result)==0: 
        print("결과가 존재하지 않습니다.")
        return False
      
      print("\n###검색 결과###")
      for i, data in df_result.iterrows():
        print(f"{i+1}위 음원: \n Track Name: {data['Track Name']} \n Artist: {data['Artist']} \n")
      return True

    except:
      print("잘못된 값을 입력하셨습니다.")
      return False


# 플레이리스트 검색
def Search_PlayList(PlayList): 
  result = list()
  if PlayList.isEmpty():
    print("플레이리스트가 비어 있습니다.")
    return result

  keyword = input("원하는 검색어를 입력해주세요. ")
    
  try:
    print("\n###검색 결과###")
    for i in range(len(PlayList.array)):
      row = PlayList.array[i]
      if (keyword.upper() in row[0].upper()) or (keyword.upper() in row[1].upper()):
        result.append([i+1, row[0], row[1]])
        print(f"{i+1}번째 음원: \n Track Name: {row[0]} \n Artist: {row[1]} \n")

    if len(result)==0: 
      print("결과가 존재하지 않습니다.")
      return False
    return True

  except:
    print("잘못된 값을 입력하셨습니다.")
    return False


# 플레이리스트에 음원 삽입
# 0 누를 때까지 반복하여 삽입
def Create(Ranking, PlayList):
  while True:
    try:
      idx = int(input("넣고 싶은 곡의 번호를 입력해주세요(1~200, 0: 종료). "))
      if idx == 0:
        print("\ninsert 완료\n")
        break
      result = PlayList.enqueue(list(Ranking.iloc[idx-1, :]))
      print("\n###삽입 목록###")
      print(f" Track Name: {result[0]} \n Artist: {result[1]} \n")
    except:
      print("잘못된 값을 입력하셨습니다.")


# 입력한 횟수만큼 플레이리스트 전체 재생
def Play(PlayList): 
  # 플레이리스트가 비어 있으면, 종료
  if PlayList.isEmpty():
    print("플레이 리스트가 비어 있습니다.")
    return
  
  try:
    play = int(input("재생 횟수를 입력해주세요. "))
    for i in range(play):
      idx = PlayList.front
      print(f"\n###{i+1}번째 재생###")
      while True:
        if idx == PlayList.rear:
          break
        tmp = PlayList.array[idx]
        print(f" Track Name: {tmp[0]} \n Artist: {tmp[1]} \n")
        idx = idx + 1
  except:
    print("잘못된 값을 입력하셨습니다.")


# 플레이리스트를 검색하여 제거
def Delete_By_Search(PlayList):
  # 플레이리스트가 비어 있으면, 종료
  if PlayList.isEmpty():
    print("플레이리스트가 비어 있습니다.")
    return
  
  try:
    result = Search_PlayList(PlayList)
    if not result:
      return
    
    n = int(input("몇 번째 노래를 삭제할까요? "))

    tmp = PlayList.dequeue(n-1)
    print("\n###삭제 목록###")
    print(f"\n Track Name: {tmp[0]} \n Artist: {tmp[1]} \n")
    print("delete 완료")
  except:
    print("잘못된 값을 입력하셨습니다.")
    return

## 2.4. user_main

In [8]:
##### 이용자 모드: 실행 #####
def user_main():
  print("*"*30)
  print(" Ranking 데이터 생성 중... \n 잠시만 기다려 주세요")
  print("*"*30)
  ranking = Create_Ranking()
  
  while (1):
    print("*"*30)
    print(" 1: 전체 랭킹 출력 \n 2: 랭킹 검색 \n 3: 플레이리스트 검색 \n 4: 추가 \n 5: 재생 \n 6: 제거 \n q: 종료")
    print("*"*30)

    function = input("무엇을 하고자 하십니까? ")
    if function == "q":
      break
    elif function == "1":
      Display_Ranking(ranking)
    elif function == "2":
      Search_Ranking(ranking)
    elif function == "3":
      Search_PlayList(play_list)
    elif function == "4":
      Create(ranking, play_list)
    elif function == "5":
      Play(play_list)
    elif function == "6":
      Delete_By_Search(play_list)

    else:
      print("잘못된 입력입니다.")

# 실행

In [9]:
# ---------------------------------------- #
# ----------------- 실행 ----------------- #
# ---------------------------------------- #

DB = Heap(df)
play_list = PlayList()

def main():
  while (1):
    print("*"*30)
    print(" A: 관리자 모드 \n U: 이용자 모드 \n Q: 종료")
    print("*"*30)

    function = input("모드 선택. ")

    if function == "Q":
      break
    elif function == "A":
      admin_main()
    elif function == "U":
      user_main()
    
    else:
      print("잘못된 입력입니다.")

In [10]:
main()

******************************
 A: 관리자 모드 
 U: 이용자 모드 
 Q: 종료
******************************
모드 선택. A
******************************
 1: 추가 
 2: 업데이트 
 3: 검색 
 4: 제거 
 5: 출력 
 q: 종료
******************************
무엇을 하고자 하십니까? 1
노래 제목을 입력해주세요. Let it go
가수 이름을 입력해주세요. Elsa
스트리밍 수를 입력해주세요. 1000000

###삽입 결과###

 
 Track Name: Let it go
 Artist: Elsa 
 Streams: 1000000 


insert 완료

******************************
 1: 추가 
 2: 업데이트 
 3: 검색 
 4: 제거 
 5: 출력 
 q: 종료
******************************
무엇을 하고자 하십니까? 2
원하는 검색어를 입력해주세요. love

###검색 결과###
23번째 음원: 
 Track Name: What You Know Bout Love 
 Artist: Pop Smoke 

31번째 음원: 
 Track Name: Someone You Loved 
 Artist: Lewis Capaldi 

64번째 음원: 
 Track Name: Savage Love (Laxed - Siren Beat) 
 Artist: Jawsh 685 

74번째 음원: 
 Track Name: lovely (with Khalid) 
 Artist: Billie Eilish 

105번째 음원: 
 Track Name: Love Not War (The Tampa Beat) 
 Artist: Jason Derulo 

156번째 음원: 
 Track Name: ily (i love you baby) (feat. Emilee) 
 Artist: Surf Mesa 

179번째 음원